In [84]:
import pandas as pd
import pickle
import numpy as np

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [11]:
test_data = pd.read_csv("/content/drive/MyDrive/Applied machine learning/project_files/m1_test.csv")
train_data = pd.read_csv("/content/drive/MyDrive/Applied machine learning/project_files/m1_train.csv")

# Loading cluster data. cluster_data is a dictionary with the key being cluster label and the values being members of the cluster
with open('/content/drive/MyDrive/Applied machine learning/project_files/cluster_data.pkl', 'rb') as cluster_file:
    cluster_data = pickle.load(cluster_file)

In [75]:
train_df = train_data.groupby('customer_id')['article_id'].agg(previous_buy_size = len, previous_buy= lambda x: list(x)).reset_index()
train_df

,customer_id,previous_buy_size,previous_buy
0,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,3,"[471714005, 471714002, 471714008]"
1,001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7a...,1,[573390001]
2,0019c05146d30111b7003700c712f9354fb62f9b87e53b...,18,"[688873021, 547367010, 547365012, 611415005, 7..."
3,00227494dd4e87da02bb1ab4afc38f13f2e11c6517b1bb...,84,"[617036003, 572325004, 611020008, 692200001, 6..."
4,0036414a83662a899a2e2c30854d8224932cc2e0d6f9c5...,32,"[667319001, 640029001, 648719001, 705143001, 7..."
...,...,...,...
12301,ffd8a6ce04a08854dc847409f7b9f1506f4509482ef73c...,8,"[720137004, 720137003, 720137001, 621381001, 7..."
12302,ffdae68f247ef0c78da5bf053dd68b7b4e9aeeec8135bb...,1,[518158002]
12303,ffe44295c63a13498687134a9e6ee5c57e08d84bfffa0d...,17,"[744394004, 747939002, 749699002, 688537011, 6..."
12304,ffeb2a74486809e237cd05be056ca2e33b2141172e99d3...,13,"[758126002, 758084002, 369796035, 594978031, 5..."


In [76]:
test_df = test_data.groupby('customer_id')['article_id'].agg(size = len, actual_buy= lambda x: list(x)).reset_index()
test_df

,customer_id,size,actual_buy
0,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,1,[471714014]
1,001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7a...,1,[573390001]
2,0019c05146d30111b7003700c712f9354fb62f9b87e53b...,5,"[608776020, 778064005, 772773002, 772773001, 7..."
3,00227494dd4e87da02bb1ab4afc38f13f2e11c6517b1bb...,21,"[740812007, 399223001, 554450025, 399223033, 7..."
4,0036414a83662a899a2e2c30854d8224932cc2e0d6f9c5...,9,"[860885002, 873019001, 806388005, 886966001, 8..."
...,...,...,...
12301,ffd8a6ce04a08854dc847409f7b9f1506f4509482ef73c...,3,"[757711001, 854004001, 640021012]"
12302,ffdae68f247ef0c78da5bf053dd68b7b4e9aeeec8135bb...,1,[501722005]
12303,ffe44295c63a13498687134a9e6ee5c57e08d84bfffa0d...,5,"[698286007, 698286007, 699453001, 699453001, 6..."
12304,ffeb2a74486809e237cd05be056ca2e33b2141172e99d3...,4,"[685816008, 685816008, 685816008, 685816008]"


In [51]:
# Function to list of recommendations based on list of previous purchases
def get_recc(item_list):
  recc_cluster = []
  recc_buy = []
  for item in item_list:
    item = str(0) + str(item) + str('.jpg') # Required for the format of filename of corresponsing image file
    for key in cluster_data.keys():
      if item in cluster_data[key] and key not in recc_cluster:
        recc_cluster += [key]
        recc_buy += cluster_data[key]
    try:
      recc_buy.remove(item)
    except:
      pass
  
  return recc_buy
  

In [60]:
# Using previous buy column to generate recommendations for each customer
recc_buy = []
count=0
for i in train_df['previous_buy']:
  recc_buy.append(get_recc(i))
  count+=1
  print(f'\r{count} customers have been processed', end='', flush=True)


12306 customers have been processed

In [82]:
truth_df = pd.DataFrame()
truth_df['customer_id'] = test_df['customer_id']
truth_df['actual_buy'] = test_df['actual_buy']
truth_df['recc_buy'] = recc_buy

for index,buy_list in enumerate(truth_df['actual_buy']):
  new_buy_list=[]
  for item in buy_list:
    new_buy_list+=[str(0)+str(item)+str('.jpg')] # Required for the format of filename of corresponding image file 
  truth_df['actual_buy'][index] = new_buy_list

In [83]:
truth_df

,customer_id,actual_buy,recc_buy
0,000362878a3904e1fe4927bbfcdb10c64a9d85b12a593a...,[0471714014.jpg],"[0399201020.jpg, 0453239044.jpg, 0453258007.jp..."
1,001270f9c6b9e0fe78121ca9d9071f4b9078da72258a7a...,[0573390001.jpg],"[0322017043.jpg, 0468591020.jpg, 0513984011.jp..."
2,0019c05146d30111b7003700c712f9354fb62f9b87e53b...,"[0608776020.jpg, 0778064005.jpg, 0772773002.jp...","[0296476008.jpg, 0296476014.jpg, 0296476015.jp..."
3,00227494dd4e87da02bb1ab4afc38f13f2e11c6517b1bb...,"[0740812007.jpg, 0399223001.jpg, 0554450025.jp...","[0249136022.jpg, 0249136034.jpg, 0254940026.jp..."
4,0036414a83662a899a2e2c30854d8224932cc2e0d6f9c5...,"[0860885002.jpg, 0873019001.jpg, 0806388005.jp...","[0561776004.jpg, 0616540002.jpg, 0616540003.jp..."
...,...,...,...
12301,ffd8a6ce04a08854dc847409f7b9f1506f4509482ef73c...,"[0757711001.jpg, 0854004001.jpg, 0640021012.jpg]","[0506191003.jpg, 0548231005.jpg, 0548231011.jp..."
12302,ffdae68f247ef0c78da5bf053dd68b7b4e9aeeec8135bb...,[0501722005.jpg],"[0212629032.jpg, 0299733007.jpg, 0396135023.jp..."
12303,ffe44295c63a13498687134a9e6ee5c57e08d84bfffa0d...,"[0698286007.jpg, 0698286007.jpg, 0699453001.jp...","[0458543001.jpg, 0458543007.jpg, 0492245002.jp..."
12304,ffeb2a74486809e237cd05be056ca2e33b2141172e99d3...,"[0685816008.jpg, 0685816008.jpg, 0685816008.jp...","[0451744005.jpg, 0451744006.jpg, 0451744018.jp..."


In [90]:
def apk2(actual, predicted):
  num_hits = 0.0

  for i,p in enumerate(actual):
    if p in predicted:
      num_hits += 1.0

  return num_hits / len(actual)

def mapk2(df):
  s = []
  for idx, row in df.iterrows():
    s.append(apk2(row['actual_buy'], row['recc_buy']))
  return np.mean(s)

In [91]:
print(f'Mean Average Precision @k = {mapk2(truth_df)}')

Mean Average Precision @k = 0.10100720495323853
